In [13]:
%load_ext autoreload
%autoreload 2
import pdfplumber, camelot, pprint, json5
import fitz #type: ignore
import json
from pathlib import Path
import pandas as pd
import numpy as np
import json, os, re

with open("config\\sid_params.json5","r+") as file:
    params = json5.load(file)

# with open("config\\sid_params.json5","r+") as file:
#     params = json5.dump(params,file)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
# | Tool      | Page Indexing | Example                  |
# | --------- | ------------- | ------------------------ |
# | `fitz`    | **0-based**   | `doc[0]` → first page    |
# | `camelot` | **1-based**   | `pages="1"` → first page |


In [129]:
path = os.path.join(os.getcwd(),"pdfs","bajaj1727439432823.pdf")
path2 = os.path.join(os.getcwd(),"pdfs","SID-Angel-One-Nifty-50-Index-Fund-2.pdf")

In [ ]:
all_data = []
with fitz.open(path) as doc:
    for i, page in enumerate(doc):
        if i in [0]:
            text = page.get_text("text",clip =(5, 125, 190, 455)).split("\n")
            for t in text:
                all_data.append(t)
# pprint.pprint(all_data)

In [ ]:
def _parse_page_zero(path:str,amc:str,pages:list):
    final_dict = {}
    bbox_text = []
    other_text = []
    risk_bbox = params[amc]["PARAMS"]["page_zero"]["bbox"]
    with fitz.open(path) as doc:
        for pgn, page in enumerate(doc):
            if pgn in pages:
                bbox_text.append(page.get_text("text", clip=risk_bbox).replace("\n", ""))
                other_text.append(page.get_text("text").replace("\n", ""))
        final_dict["risk_bbox"] = bbox_text
        final_dict["other_text"] = other_text
    return final_dict

def _detect_column_start_by_keywords(df,threshold,keywords): 
    for i in range(df.shape[1]):
        col = df.iloc[:, i]
        col_cleaned = col.dropna().astype(str)
        col_cleaned = col_cleaned.apply(lambda x: re.sub(r"\s+", " ", x).strip())
        col_cleaned = col_cleaned[col_cleaned.str.strip() != ""]

        count = sum(any(kw in cell for kw in keywords) for cell in col_cleaned)

        if count >= threshold:
            return i
    return 1

def _parse_scheme_table_data(path:str,amc:str,pages:str):
    final_dict = {}
    
    threshold = params[amc]["PARAMS"]["table_data"]["threshold"]
    keywords = params[amc]["PARAMS"]["table_data"]["keywords"]
    
    scheme_tables = camelot.read_pdf(path, pages=pages, flavor='lattice')
    
    dfs = pd.concat([table.df for table in scheme_tables], ignore_index=True)
    col_start = _detect_column_start_by_keywords(dfs,threshold=threshold,keywords=keywords)
    dfs = dfs.iloc[:,col_start:]
    # print(dfs.head(20))
    dfs = dfs.applymap(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x) #applies to dataframe not series
    dfs.iloc[:, 0] = dfs.iloc[:, 0].replace(r'^\s*$', np.nan, regex=True)
    dfs.iloc[:, 0] = dfs.iloc[:, 0].ffill() #fill corresponding structure
    dfs.columns = ['Title'] + [f'Data{i}' for i in range(1, dfs.shape[1])] #new col structure
    
    data_cols = [col for col in dfs.columns if col != 'Title']
    for title, group_df in dfs.groupby('Title', sort=False):
        title = re.sub(r"[^A-Za-z0-9\s]+","",title).strip()
        title = re.sub(r"\s+"," ",title).strip()
        title = "_".join(title.lower().split(" "))
        
        current_title_values = []
        for index, row in group_df.iterrows():
            for col_name in data_cols:
                cell_value = row[col_name]
                if isinstance(cell_value, str) and cell_value.strip() != '':
                    current_title_values.append(cell_value)
        final_dict[title] = current_title_values
        
    return final_dict

def _parse_fund_manager_info(path:str,amc:str,pages:str):
    
    row_count = params[amc]["PARAMS"]["manager_data"]["row_count"]
    match_order = params[amc]["PARAMS"]["manager_data"]["order"]
    manager_tables = camelot.read_pdf(path, pages=pages, flavor='lattice')
    dfs = pd.concat([table.df for table in manager_tables], ignore_index=True)
    dfs = dfs.applymap(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x)
    
    data_rows = [list(row) for _, row in dfs.iterrows() if row.count()==row_count]
    manager_list = []
    for data_r in data_rows:
        manager_list.append({
            "name":data_r[match_order["name"]],
            "experience":data_r[match_order["experience"]],
            "qualification":data_r[match_order["qualification"]],
        })
    return  {"fund_manager": manager_list}

In [ ]:
temp_dict = {}
temp_dict.update(_parse_page_zero(path,"59",[0]))
temp_dict.update(_parse_scheme_table_data(path,"59","5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21"))
temp_dict.update(_parse_fund_manager_info(path,"59","29,30"))

In [ ]:
temp_dict = {}
temp_dict.update(_parse_page_zero(path2,"96",[0]))
temp_dict.update(_parse_scheme_table_data(path2,"96","5,6,7,8,9,10"))
temp_dict.update(_parse_fund_manager_info(path2,"96","16,17"))

In [162]:
with open("tempor.json","w+") as file:
    json.dump(temp_dict,file)

MANAGER